In [1]:
####
# Original network adapted from karpathy
# minesh.mathew@gmail.com
# modified version of text generation example in keras;
# trained in a many-to-many fashion using a time distributed dense layer

####

from matplotlib import pyplot as plt
from matplotlib import pyplot
import matplotlib as mptl
import pylab

import tensorflow as tf
from tensorflow import keras as keras




from __future__ import print_function
#import keras
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
# from keras.layers import LSTM, TimeDistributedDense, SimpleRNN  #DEPRECATED TimeDistributedDense
from keras.layers import LSTM, TimeDistributed, SimpleRNN
from keras.utils.data_utils import get_file
import numpy as np
from time import sleep
import random
import sys
import os
import pickle
import gzip
from sklearn.metrics import confusion_matrix
import itertools
#import h5py

In [2]:
#Test imports & GPU
print("Tensorflow version: ", tf.__version__)
print("Tensorflow devices: ", tf.config.list_physical_devices("GPU"))
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("GPUs Available: ", tf.config.experimental.list_physical_devices('GPU'))

Tensorflow version:  2.6.0
Tensorflow devices:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Num GPUs Available:  1
GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [4]:
# ---------------------- GENERIC UTILITIES --------------------------
def plot_confusion_matrix(cm, class_labels, title='Confusion matrix',
                          filename = 'Confusion_Matrix.png', cmap=plt.cm.Blues):
    """
    Plot a confusion matrix
    :param cm: a confusion matrix generated by sklearn.metrics.confusion_matrix
    :param class_labels: set of text labels
    :param title: title of figure
    :param cmap: color map for the confusion matrix
    :return:
    """
    class_count = len(class_labels)
    fig = plt.figure(title)
    fig.set_size_inches(10, 8)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(class_count+1)
    plt.xticks(tick_marks, class_labels, rotation=45)
    plt.yticks(tick_marks, class_labels)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j], horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")
    #plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig(filename, dpi=100)
    np.set_printoptions(precision=2)
    plt.show(block=False)

In [5]:
#-------------------- TEXT FILE PREPROCESSING -----------------

def preprocess_text_file(filename, maxlen=40):
    print('loading: ', filename)
    text = open(filename).read().lower()
    print('corpus length:', len(text))
    chars = sorted(list(set(text)))  #returns unique characters from the text
    print('total chars:', len(chars))
    char_indices = dict((c, i) for i, c in enumerate(chars))
    indices_char = dict((i, c) for i, c in enumerate(chars))
    # split the corpus into sequences of length=maxlen
    # input is a sequence of 40 chars and target is also a sequence of 40 chars shifted by one position
    # for eg: if you maxlen=3 and the text corpus is abcdefghi, your input ---> target pairs will be
    # [a,b,c] --> [b,c,d], [b,c,d]--->[c,d,e]....and so on
    step = 1
    sentences = []
    next_chars = []
    #first generate sentences of characters
    for i in range(0, len(text) - maxlen + 1, step):
        sentences.append(text[i: i + maxlen])  # input seq is from i to i  + maxlen
        next_chars.append(text[i + 1:i + 1 + maxlen])  # output seq is from i+1 to i+1+maxlen
        # if i<10 :
        # print (text[i: i + maxlen])
        # print(text[i+1:i +1+ maxlen])
    print('number of sequences:', len(sentences))
    return text, chars, char_indices, indices_char, sentences, next_chars


def save_processed_text(filename, text, chars, char_indices, indices_char, sentences, next_chars ):
    #helper method for saving processed text in a compressed file
    my_text = [text, chars, char_indices, indices_char, sentences, next_chars] #dont save the one-hot vectors
    with gzip.open(filename, 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump(my_text, f)
    # np.savez_compressed(filename,
    #                     text = text,
    #                     chars = chars,
    #                     char_indices=char_indices,
    #                     indices_char=indices_char,
    #                     sentences=sentences,
    #                     next_chars=next_chars)
    #                     # allow_pickle=True, protocol=4)


def load_processed_text(filename):
    #helper method for saving processed text in a compressed file
    with gzip.open(filename, 'rb') as f:  # Python 3: open(..., 'rb')
        text, chars, char_indices, indices_char, sentences, next_chars = pickle.load(f)
    # loaded = np.load(filename)
    # text = loaded['text']
    # chars = loaded['chars']
    # char_indices= loaded['char_indices']
    # indices_char= loaded['indices_char']
    # sentences= loaded['sentences']
    # next_chars= loaded['next_chars']
    return text, chars, char_indices, indices_char, sentences, next_chars


def vectorize_text(chars, char_indices, sentences, next_chars, maxlen=40):

    # now generate dummy variables (1-hot vectors) for the sequences of characters
    print('Vectorization processing... this could take a while...')
    X = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
    y = np.zeros((len(sentences), maxlen, len(chars)),
                 dtype=bool)  # y is also a sequence , or  a seq of 1 hot vectors
    joblength = len(sentences)
    tenpercent = joblength/10
    nextpercent = tenpercent
    print(" part 1 of 2")
    for i, sentence in enumerate(sentences):
        if i>nextpercent:
            print(i, " of ", joblength, " completed")
            nextpercent += tenpercent
        for t, char in enumerate(sentence):
            X[i, t, char_indices[char]] = 1  # X has dimension [sentence_count, sentence_length, char_count]
    print(" part 2 of 2")
    nextpercent = tenpercent

    for i, sentence in enumerate(next_chars):
        if i>nextpercent:
            print(i, " of ", joblength, " completed")
            nextpercent += tenpercent
        for t, char in enumerate(sentence):
            y[i, t, char_indices[char]] = 1  # y has dimension [sentence_count, sentence_length, char_count]
    print('vetorization completed')
    return X, y


def generate_text(model, char_indices, indices_char, seed_string="brutus:", generate_character_count=320):

    print("seed string --> ", seed_string)
    print('The generated text is: ')
    sys.stdout.write(seed_string),
    # x=np.zeros((1, len(seed_string), len(chars)))
    for i in range(generate_character_count):
        x = np.zeros((1, len(seed_string), len(chars)))
        for t, char in enumerate(seed_string):
            x[0, t, char_indices[char]] = 1.
        preds = model.predict(x, verbose=0)[0]
        # print (np.argmax(preds[7]))
        next_index = np.argmax(preds[len(seed_string) - 1])

        # next_index=np.argmax(preds[len(seed_string)-11])
        # print (preds.shape)
        # print (preds)
        # next_index = sample(preds, 1) #diversity is 1
        next_char = indices_char[next_index]
        seed_string = seed_string + next_char

        # print (seed_string)
        # print ('##############')
        # if i==40:
        #    print ('####')
        sys.stdout.write(next_char)
    sys.stdout.flush()

In [6]:
# ---------------- MODEL FILE I/O ---------------------------
def save_model(model, save_dir=os.path.join(os.getcwd(), 'saved_models'),
               model_file_name='keras_cifar10_trained_model.h5'):
    """
    Save model and current weights
    :param model: Keras model
    :param save_dir: path name to save directory
    :param model_file_name: filename for saved model
    :return: nothing
    """
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    model_path = os.path.join(save_dir, model_file_name)
    model.save(model_path)
    print('Saved trained model at %s ' % model_path)


def load_model(save_dir, model_file_name):
    # Load model and weights
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    model_path = os.path.join(save_dir, model_file_name)
    model = keras.models.load_model(model_path)
    print('Loaded trained model from %s ' % model_path)
    return model

In [7]:
# ---------------- MODEL DEVELOPMENT ---------------------------
def build_model(characters):
    # build the model: 2 stacked LSTM
    print('Build model...')
    model = Sequential()
    # model.add(LSTM(512, return_sequences=True, input_shape=(maxlen, len(chars))))  # original one
    model.add(LSTM(512, input_dim=len(characters), return_sequences=True))  # minesh witout specifying the input_length
    model.add(LSTM(512, return_sequences=True))  # - original
    model.add(Dropout(0.2))
    # model.add(TimeDistributedDense(len(chars)))   #Deprecated TimeDistributedDense
    model.add(TimeDistributed(Dense(len(characters))))  # BJB:  is this really working??
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    print('model is made')
    # train the model, output generated text after each iteration
    print(model.summary())
    return model


def train_net(model, x, y, training_iterations=6, maxlen=40, save_all_model_iterations=True):
    for training_iteration in range(1, training_iterations+1):
        print()
        print('-' * 50)
        print('Training Iteration (epoch) #:', training_iteration)
        history = model.fit(x, y, batch_size=128, epochs=1, verbose=1)    #train 1 epoch at a time using previous weights
        sleep(0.1)  # https://github.com/fchollet/keras/issues/2110

        # saving models at the following iterations -- uncomment it if you want tos save weights and load it later
        # if training_iteration==1 or training_iteration==3 or training_iteration==5 or training_iteration==10 or training_iteration==20 or training_iteration==30 or training_iteration==50 or training_iteration==60 :

        # # save every training_iteration of weights
        # model.save_weights('Karpathy_LSTM_weights_' + str(training_iteration) + '.h5', overwrite=True)
        # start_index = random.randint(0, len(text) - maxlen - 1)
        save_dir = os.path.join(os.getcwd(), 'saved_models')
        current_model_file_name = 'LSTM_model_' + str(training_iteration) + '.h5'
        if save_all_model_iterations:
            save_model(model=model, save_dir=save_dir, model_file_name=current_model_file_name)
        sys.stdout.flush()
        print('loss is')
        print(history.history['loss'][0])
        print(history)
        print()
    return model

In [8]:
# ---------------- MODEL EVALUATION ---------------------------
def test_model(model, observations, targets):
    '''
    STUDENT SHOULD WRITE THIS CODE
    :param model: a trained RNN model which accepts a sequence and outputs a target class (0;1;2;3)
    :param observations: a list of 40-character sequences to classify
    :param targets: a list of the true classes of the 40-character sequences
    :return: a sklearn confusion matrix
    '''
    #< put student code here to test the model >
    actual_class_IDs = []
    predicted_class_IDs = []
    # generate & print confusion matrix to screen
    cm = confusion_matrix(actual_class_IDs, predicted_class_IDs)
    return cm

In [9]:
################################################################
#--------------------- Main Code -----------------------------


# pick the filename you want to use, and comment out the rest
# make sure you have this directory structure
# QUICK CHECK w/SMALL TEXT
raw_text_filename='./textdatasets/tinytesttext.txt'
# CLASS DATASETS BELOW
# raw_text_filename='./textdatasets/0_bible.txt'
# raw_text_filename='./textdatasets/1_nietzsche.txt'
# raw_text_filename='./textdatasets/2_shakespeare.txt'
# raw_text_filename='./textdatasets/3_warpeace.txt'



#raw_text_filename='./textdatasets/trumptweets.txt'

processed_filename = raw_text_filename+'.pklz'  # save process will append a .pklz on the filename

TEST_SAVE_LOAD_EQUAL = False
if TEST_SAVE_LOAD_EQUAL:
    text, chars, char_indices, indices_char, sentences, next_chars \
        = preprocess_text_file(raw_text_filename)
    save_processed_text(processed_filename, text, chars, char_indices, indices_char, sentences, next_chars )
    _text, _chars, _char_indices, _indices_char, _sentences, _next_chars = load_processed_text(processed_filename)
    print(np.array_equal(_text, text))
    print(np.array_equal(_chars, chars))
    print(np.array_equal(_char_indices, char_indices))
    print(np.array_equal(_indices_char, indices_char))
    print(np.array_equal(_sentences, sentences))
    print(np.array_equal(_next_chars, next_chars))
    print("testing vectorization")
    X, y = vectorize_text(_chars, _char_indices, _sentences, _next_chars)

PROCESS_RAW_TEXT = True  #set to True to process a previously unseen textfile - otherwise load a preprocessed file
if PROCESS_RAW_TEXT:
    text, chars, char_indices, indices_char, sentences, next_chars \
        = preprocess_text_file(raw_text_filename)
    save_processed_text(processed_filename, text, chars, char_indices, indices_char, sentences, next_chars)
else:  # instead, load previously processed text
    text, chars, char_indices, indices_char, sentences, next_chars = load_processed_text(processed_filename)


#vectorized form takes too much space to save... so process in real time
X, y = vectorize_text(chars, char_indices, sentences, next_chars)


TRAIN_MODE = True
if TRAIN_MODE:
    model = build_model(characters=chars)
    model_epoch_training_iterations = 20    #the bigger your text corpus, the smaller you can make this
    model = train_net(model=model, x=X, y=y,
                      training_iterations=model_epoch_training_iterations,
                      save_all_model_iterations=True)
else:  # load a model from a file
    # decide which iteration of the trained model you want to explore
    model_training_iteration = 8
    save_dir = os.path.join(os.getcwd(), 'saved_models')
    current_model_file_name = 'LSTM_model_' + str(model_training_iteration) + '.h5'
    model = load_model(save_dir=save_dir, model_file_name=current_model_file_name)

GENERATE_TEXT_MODE = True
if GENERATE_TEXT_MODE:   # generate text mode
    #decide which saved model to load
    #make up a string of characters to start with
    seed_string = "certainly"
    # decide how many text characters you want to generate:
    gen_char_count = 200
    generate_text(model, char_indices, indices_char, seed_string, generate_character_count=gen_char_count)

loading:  ./textdatasets/tinytesttext.txt
corpus length: 5601
total chars: 43
number of sequences: 5562
Vectorization processing... this could take a while...
 part 1 of 2
557  of  5562  completed
1113  of  5562  completed
1669  of  5562  completed
2225  of  5562  completed
2782  of  5562  completed
3338  of  5562  completed
3894  of  5562  completed
4450  of  5562  completed
5006  of  5562  completed
 part 2 of 2
557  of  5562  completed
1113  of  5562  completed
1669  of  5562  completed
2225  of  5562  completed
2782  of  5562  completed
3338  of  5562  completed
3894  of  5562  completed
4450  of  5562  completed
5006  of  5562  completed
vetorization completed
Build model...
model is made
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, None, 512)         1138688   
_________________________________________________________________
lstm_1 (LSTM)